# Подгонка ARIMA (пакет [`sktime`](https://www.sktime.net/en/stable/))

In [ ]:
import numpy as np
import pandas as pd

from sktime.forecasting.arima import ARIMA, AutoARIMA
from sktime.utils.plotting import plot_series

import pandas_datareader.data as web

# тесты диагностики
from statsmodels.stats.diagnostic import het_arch, acorr_ljungbox

# настройки визуализация
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)
# Не показывать ValueWarning, ConvergenceWarning из statsmodels
from statsmodels.tools.sm_exceptions import ValueWarning, ConvergenceWarning
warnings.simplefilter('ignore', category=ValueWarning)
warnings.simplefilter('ignore', category=ConvergenceWarning)

## Подгонка ARIMA заданного порядка без сноса

Загрузим из БД [`FRED`](https://fred.stlouisfed.org/) недельные данные по 3-Month Treasury Bill Secondary Market Rate (Symbol [`WTB3MS`](https://fred.stlouisfed.org/series/WTB3MS)) с 2000-01-01 по 2023-12-31 и создадим датафрейм `rate`

In [ ]:
y = web.DataReader(name='WTB3MS', data_source='fred', start='2000-01-01', end='2023-12-31')

Подгоним модель ARMA(2,1,2) без сноса для `y`

Спецификация

$$
	(1-\phi_1L-\phi_2 L^2)(1-L) y_t=u_t+\theta_1 u_{t-1}+\theta_2 u_{t-2}
$$

In [ ]:
forecaster = ARIMA(order=(2,1,2), trend='n')
forecaster.fit(y)
forecaster.summary()

In [ ]:
forecaster.get_fitted_params()

## Диагностика модели

Остатки (__обратим внимание на пропущенные значения!__)

In [ ]:
forecaster.predict_residuals(y)

Тест на серийную корреляцию (Ljung-Box)

__Важно__ удалим пропущенные значения для остатков

In [ ]:
# model_df = p+q
acorr_ljungbox(forecaster.predict_residuals(y).dropna() , lags=[7], model_df=2+2)

Тест на гетероскедастичность

__Важно__ удалим пропущенные значения для остатков

In [ ]:
lm_stat, lm_pval, f_stat, f_pval = het_arch(forecaster.predict_residuals(y).dropna(), nlags=7, ddof=2+2)

lm_stat, lm_pval

## Подгонка ARIMA заданного порядка со сносом

Загрузим из БД [`FRED`](https://fred.stlouisfed.org/) квартальные данные по ВВП (Symbol [`GDP`](https://fred.stlouisfed.org/series/GDP)) с 1990 Q1 по 2023 Q4 и создадим датафрейм `y=log(GDP)`

In [ ]:
y = np.log(web.DataReader(name='GDP', data_source='fred', start='1990-1', end='2023-12' ) )

Подгоним модель ARMA(1,1,1) со сносом для `y` 

Спецификация

$$
	(1-\phi L)(1-L) y_t=\alpha_0+u_t+\theta u_{t-1}
$$

In [ ]:
forecaster = ARIMA(order=(1,1,1), trend='c')
forecaster.fit(y)
forecaster.summary()

## Диагностика

In [ ]:
# model_df = p+q+1 (drift)
acorr_ljungbox(forecaster.predict_residuals(y).dropna() , lags=[7], model_df=1+1+1)

In [ ]:
# model_df = p+q+1 (drift)
lm_stat, lm_pval, f_stat, f_pval = het_arch(forecaster.predict_residuals(y).dropna(), nlags=7, ddof=1+1+1)

lm_stat, lm_pval

# Автоматический выбор порядка модели

Выбор порядка по информационным критериям. Выведем только порядок подогнанной модели

In [ ]:
forecaster = AutoARIMA(start_p=1, start_q=1, max_p=5, max_q=5, information_criterion='aic', seasonal=False)
forecaster.fit(y)
forecaster.get_fitted_params() #['order']

In [ ]:
forecaster = AutoARIMA(start_p=1, start_q=1, max_p=5, max_q=5, information_criterion='bic', seasonal=False)
forecaster.fit(y)
forecaster.get_fitted_params()['order']

In [ ]:
forecaster = AutoARIMA(start_p=1, start_q=1, max_p=5, max_q=5, information_criterion='bic', seasonal=False)
forecaster.fit(y)
forecaster.get_fitted_params()['order']